# Swaps

In [ ]:
#| default_exp swap

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from sugar.quote import Quote, pack_path
from sugar.helpers import apply_slippage
from sugar.pool import LiquidityPoolForSwap
from enum import IntEnum
from typing import List, Any, Tuple
from eth_abi import encode

In [ ]:
#| export

class CommandType(IntEnum):
    V3_SWAP_EXACT_IN = 0x00
    V3_SWAP_EXACT_OUT = 0x01
    SWEEP = 0x04
    V2_SWAP_EXACT_IN = 0x08
    V2_SWAP_EXACT_OUT = 0x09
    WRAP_ETH = 0x0b
    UNWRAP_WETH = 0x0c
    TRANSFER_FROM = 0x07
    BRIDGE_TOKEN = 0x12
    EXECUTE_CROSS_CHAIN = 0x13
    EXECUTE_SUB_PLAN = 0x21

class BridgeType(IntEnum):
    """
    Enum for different bridge types.
    """
    HYP_XERC20 = 0x01
    XVELO = 0x02

FLAG_ALLOW_REVERT = 0x80

# Define ABI types for each command
ABI_DEFINITION = {
    CommandType.V3_SWAP_EXACT_IN: [
        "address", # recipient
        "uint256", # amountIn
        "uint256", # amountOutMin
        "bytes", # path
        "bool", # payerIsUser
        "bool"  # isUni
    ],
    CommandType.V2_SWAP_EXACT_IN: [
        "address",
        "uint256",
        "uint256",
        "(address,address,bool)[]",
        "bool",
        "bool"  # isUni
    ],
    CommandType.V2_SWAP_EXACT_OUT: [
        "address",
        "uint256",
        "uint256",
        "(address,address,bool)[]",
        "bool",
        "bool"  # isUni
    ],
    CommandType.V3_SWAP_EXACT_OUT: [
        "address",
        "uint256",
        "uint256",
        "bytes",
        "bool",
        "bool"  # isUni
    ],
    CommandType.WRAP_ETH: [
        "address",
        "uint256"
    ],
    CommandType.UNWRAP_WETH: [
        "address",
        "uint256"
    ],
    CommandType.SWEEP: [
        "address",
        "address",
        "uint256"
    ],
    CommandType.TRANSFER_FROM: [
        "address",  # sender
        "address",  # recipient
        "uint256"   # amount
    ],
    CommandType.BRIDGE_TOKEN: [
        "uint8",    # bridgeType
        "address",  # recipient
        "address",  # token
        "address",  # bridge
        "uint256",  # amount
        "uint256",  # msgFee
        "uint32",   # domain
        "bool"      # payerIsUser
    ],
    CommandType.EXECUTE_SUB_PLAN: [
        "bytes",    
        "bytes[]"
    ],
    CommandType.EXECUTE_CROSS_CHAIN: [
        "uint32",   # domain
        "address",  # icaRouter
        "bytes32",  # remoteRouter
        "bytes32",  # ism
        "bytes32",  # commitment
        "uint256",  # msgFee
        "address",  # hook
        "bytes"
    ]
}

class RoutePlanner:
    def __init__(self):
        """Initialize a new RoutePlanner"""
        self.commands = "0x"
        self.inputs: List[bytes] = []
        self.bytes_commands: List[bytes] = []

    def add_command(self, command_type: CommandType, parameters: List[Any], allow_revert = False) -> None:
        """
        Add a command to the route planner
        
        Args:
            command_type: Type of command to add
            parameters: Parameters for the command
        """
        # Get the ABI definition for this command
        abi_types = ABI_DEFINITION[command_type]
        self.inputs.append(encode(abi_types, parameters))
        # Add command byte to commands
        command_type = command_type | FLAG_ALLOW_REVERT if allow_revert else command_type
        command_hex = format(command_type, '02x')
        self.commands = self.commands + command_hex
        # TODO: figure out why we keep this alongside command_hex
        self.bytes_commands.append(format(command_type, '02x'))

    def get_encoded_commands(self) -> str: return self.commands
    
    def get_encoded_inputs(self) -> List[bytes]: return self.inputs

    # using this for testing
    def get_pretty_encoded_inputs(self) -> List[str]: return list(map(lambda i: "0x" + i.hex(), self.get_encoded_inputs())) 

In [ ]:

def get_command_byte(command: int):
  return format(command, '02x')

get_command_byte(CommandType.V2_SWAP_EXACT_IN | FLAG_ALLOW_REVERT)

'88'

In [ ]:
#| export

# Constants
CONTRACT_BALANCE_FOR_V3_SWAPS = int("0x8000000000000000000000000000000000000000000000000000000000000000", 16)

def setup_planner(quote: Quote, slippage: float, account: str, router_address: str) -> RoutePlanner:
    """Setup route planner with the given quote and chain"""
    
    route_planner = RoutePlanner()
    min_amount_out = apply_slippage(quote.amount_out, slippage)

    # By default money comes from contract
    tokens_come_from_contract = False
    
    # Handle wrapped native token if needed
    if quote.from_token.wrapped_token_address:
        # When trading from native token, wrap token first
        route_planner.add_command(CommandType.WRAP_ETH, [router_address, quote.amount_in])
        tokens_come_from_contract = True
    
    # Group nodes by pool type (v2 or v3)
    grouped_nodes: List[List[Tuple[LiquidityPoolForSwap, bool]]] = []
    
    # helpers for getting from and to token addresses
    def from_token_address(node: Tuple[LiquidityPoolForSwap, bool]) -> str:
        pool, reversed = node
        return pool.token0_address if not reversed else pool.token1_address
    def to_token_address(node: Tuple[LiquidityPoolForSwap, bool]) -> str:
        pool, reversed = node
        return pool.token1_address if not reversed else pool.token0_address


    for node in quote.path:
        if not grouped_nodes: grouped_nodes.append([node])
        elif node[0].type < 1:
            # Current node is a v2 pool
            if float(grouped_nodes[-1][0][0].type) < 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
        else:
            # Current node is a v3 pool
            if grouped_nodes[-1][0][0].type >= 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
    
    if len(grouped_nodes) == 1:
        # All nodes belong to the same pool type
        nodes = grouped_nodes[0]
        is_v2_pool = float(nodes[0][0].type) < 1
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_v2_pool else CommandType.V3_SWAP_EXACT_IN,
            [
                # Where should money go?
                router_address if quote.to_token.wrapped_token_address else account,
                quote.amount_in,
                min_amount_out,
                [
                    # from, to, stable
                    (from_token_address(n), to_token_address(n), n[0].is_stable) for n in nodes
                ] if is_v2_pool else pack_path(nodes).encoded,
                not tokens_come_from_contract,
                False, # isUni
            ]
        )
    else:
        # Mixed v2 and v3 pools
        first_batch = grouped_nodes[0]
        last_batch = grouped_nodes[-1]
        rest = grouped_nodes[1:-1]
        
        # Handle first batch
        is_first_batch_v2 = not first_batch[0][0].is_cl
        next_batch = rest[0] if rest else last_batch
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_first_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
            [
                router_address if is_first_batch_v2 else next_batch[0][0].lp,
                quote.amount_in,
                0,  # No expectations on min amount out for first batch
                [
                    # from, to, stable
                    (from_token_address(n), to_token_address(n), n[0].is_stable) for n in first_batch
                ] if is_first_batch_v2 else pack_path(first_batch).encoded,
                not tokens_come_from_contract,
                False,  # isUni
            ]
        )
        
        # Handle middle batches
        for idx, batch in enumerate(rest):
            is_batch_v2 = not batch[0][0].is_cl
            next_batch = rest[idx + 1] if idx + 1 < len(rest) else last_batch
            
            route_planner.add_command(
                CommandType.V2_SWAP_EXACT_IN if is_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
                [
                    router_address if is_batch_v2 else next_batch[0][0].lp,
                    0 if is_batch_v2 else CONTRACT_BALANCE_FOR_V3_SWAPS,
                    0,  # No expectations for middle batches
                    [
                        (from_token_address(n), to_token_address(n), n[0].is_stable) for n in batch
                    ] if is_batch_v2 else pack_path(batch).encoded,
                    False,  # Money comes from contract
                    False,  # isUni
                ]
            )
        
        # # Handle last batch
        is_last_batch_v2 = not last_batch[0][0].is_cl
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_last_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
            [
                router_address if quote.to_token.wrapped_token_address else account,
                0 if is_last_batch_v2 else CONTRACT_BALANCE_FOR_V3_SWAPS,
                min_amount_out,
                [
                    (from_token_address(n), to_token_address(n), n[0].is_stable) for n in last_batch
                ] if is_last_batch_v2 else pack_path(last_batch).encoded,
                False,  # Money comes from contract
                False,  # isUni
            ]
        )
    
    # Handle unwrapping WETH if needed
    if quote.to_token.wrapped_token_address: route_planner.add_command(CommandType.UNWRAP_WETH, [account, min_amount_out])
    
    return route_planner

## Test planner

Let's test a few combinations of quote paths

In [ ]:
from fastcore.test import test_eq
from sugar.token import Token
from sugar.quote import QuoteInput

In [ ]:
account = '0x533cf9fb379488ffe0b1065c42c744fbd4b0e1a3'
router = '0x4bF3E32de155359D1D75e8B474b66848221142fc'
eth = Token(chain_id="10", chain_name="OP", token_address='ETH', symbol='ETH', decimals=18, listed=True, wrapped_token_address='0x4200000000000000000000000000000000000006')
velo = Token(chain_id="10", chain_name="OP", token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
usdc = Token(chain_id="10", chain_name="OP", token_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None)

def setup_quote(path: List[Tuple[LiquidityPoolForSwap, bool]], from_token: Token, to_token: Token, amount_in: int, amount_out: int) -> Quote:
    return Quote(input=QuoteInput(from_token=from_token, to_token=to_token, path=path, amount_in=amount_in), amount_out=amount_out)

# simple quote with 1 v2 pool
unstable_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x08')
test_eq(inputs, [
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000"
])

# v2 pool with native token wrap

unstable_v2_with_token_wrap = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=eth.wrapped_token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False)
    ],
    from_token=eth,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2_with_token_wrap, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x0b08')
test_eq(inputs, [
    "0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005",
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000042000000000000000000000000000000000000060000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000"
])

# v2 with native token unwrap

unstable_v2_with_token_unwrap = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=eth.wrapped_token_address, is_stable=False, is_cl=False), False)
    ],
    from_token=velo,
    to_token=eth,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2_with_token_unwrap, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x080c')
test_eq(inputs, [
    "0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db00000000000000000000000042000000000000000000000000000000000000060000000000000000000000000000000000000000000000000000000000000000",
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a3000000000000000000000000000000000000000000000000000000000000000a"
])

# simple v3 pool swap
simple_v3 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=simple_v3, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x00')
test_eq(inputs, [
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000"
])

# hybrid v2 + v2 + v3
v2_v2_v3 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v2_v2_v3, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x0800')
test_eq(inputs, [
    "0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c3160700000000000000000000000000000000000000000000000000000000000000000000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000",
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000"
])

# hybrid v2 + v3 + v2
v2_v3_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v2_v3_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x080008')
test_eq(inputs, [
    "0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000",
    "0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000",
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000"
])

# super convoluted v3 + v2 + v3 + v3 + v2
v3_v2_v3_v3_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=True), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address, is_stable=False, is_cl=False), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v3_v2_v3_v3_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x00080008')
test_eq(inputs, [
    "0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000",
    "0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000",
    "0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000429560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c316070000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000000000000000000000",
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009560e827af36c94d2ac33a39bce1fe78631088db0000000000000000000000007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000000000000000000000"
])


In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()